In [118]:
from turtle import mode
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv("emdat_disasters.csv", encoding="utf-8")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15784 entries, 0 to 15783
Data columns (total 46 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   DisNo.                                     15784 non-null  object 
 1   Historic                                   15784 non-null  object 
 2   Classification Key                         15784 non-null  object 
 3   Disaster Group                             15784 non-null  object 
 4   Disaster Subgroup                          15784 non-null  object 
 5   Disaster Type                              15784 non-null  object 
 6   Disaster Subtype                           15784 non-null  object 
 7   External IDs                               2405 non-null   object 
 8   Event Name                                 4955 non-null   object 
 9   ISO                                        15784 non-null  object 
 10  Country               

In [119]:
def removeColumns(df):
    return df.drop(
        columns=[
            "Historic",
            "Classification Key",
            "ISO",
            "External IDs",
            "Entry Date",
            "Last Update",
            "Admin Units",
            "Disaster Subtype",
            "Origin",
            "River Basin",
            "Location",
            "End Year",
            "End Month",
            "End Day",
            "OFDA/BHA Response",
            "Appeal",
            "Declaration",
            "CPI",
            "Reconstruction Costs ('000 US$)",
            "Reconstruction Costs, Adjusted ('000 US$)",
            "Associated Types",
            "AID Contribution ('000 US$)",
            "Magnitude",
            "Magnitude Scale",
            "Latitude",
            "Longitude",
            "Start Month",
            "Start Day",
            "No. Injured",
            "No. Affected",
            "No. Homeless",
            "Total Affected",
            "Insured Damage ('000 US$)",
            "Insured Damage, Adjusted ('000 US$)",
            "Total Damage ('000 US$)",
            "Event Name",
            "Total Damage, Adjusted ('000 US$)",
        ],
        axis=1,
    )
df = df.pipe(removeColumns)

df['Total Deaths'] = df['Total Deaths'].fillna(0)

In [161]:
country_data = pd.read_csv("world-data-2023.csv", encoding="utf-8")

# Correct the names of countries in both datasets that don't match
df.loc[df['Country']=='Türkiye', 'Country'] = 'Turkey'
df.loc[df['Country']=='United States of America', 'Country'] = 'United States'
df.loc[df['Country']=='Russian Federation', 'Country'] = 'Russia'
df.loc[df['Country']=='Czechia', 'Country'] = 'Czech Republic'
df.loc[df['Country']=='Netherlands (Kingdom of the)', 'Country'] = 'Netherlands'
df.loc[df['Country']=="Democratic People's Republic of Korea", 'Country'] = 'North Korea'
df.loc[df['Country']=="Republic of Korea", 'Country'] = 'South Korea'
df.loc[df['Country']=="Congo", 'Country'] = 'Republic of the Congo'
df.loc[df['Country']=="Iran (Islamic Republic of)", 'Country'] = 'Iran'
df.loc[df['Country']=="United Republic of Tanzania", 'Country'] = 'Tanzania'
df.loc[df['Country']=="Viet Nam", 'Country'] = 'Vietnam'
df.loc[df['Country']=="Syrian Arab Republic", 'Country'] = 'Syria'
df.loc[df['Country']=="Cabo Verde", 'Country'] = 'Cape Verde'
df.loc[df['Country']=="Venezuela (Bolivarian Republic of)", 'Country'] = 'Venezuela'
df.loc[df['Country']=="Timor-Leste", 'Country'] = 'East Timor'
df.loc[df['Country']=="Bolivia (Plurinational State of)", 'Country'] = 'Bolivia'
df.loc[df['Country']=="Republic of Moldova", 'Country'] = 'Moldova'
df.loc[df['Country']=="Ireland", 'Country'] = 'Republic of Ireland'
df.loc[df['Country']=="United Kingdom of Great Britain and Northern Ireland", 'Country'] = 'United Kingdom'
df.loc[df['Country']=="Micronesia (Federated States of)", 'Country'] = 'Federated States of Micronesia'
df.loc[df['Country']=="State of Palestine", 'Country'] = 'Palestinian National Authority'
df.loc[df['Country']=="Bahamas", 'Country'] = 'The Bahamas'

# Discard countries with no match on the two datasets (doing so only discards about 362 events which is a small percentage of the dataset (~2.3%))
non_overlapping_countries = [country for country in country_data['Country'].to_list() if country not in list(df['Country'].unique())]
for country in non_overlapping_countries:
    country_data = country_data.drop(country_data[(country_data['Country'] == country)].index)

non_overlapping_countries = [country for country in list(df['Country'].unique()) if country not in country_data['Country'].to_list()]
for country in non_overlapping_countries:
    df = df.drop(df[(df['Country'] == country)].index)

# keep only desired columns
columns_to_keep = ['Country', 'Land Area(Km2)']
country_data = country_data[columns_to_keep]
country_data['Land Area(Km2)'] = country_data['Land Area(Km2)'].str.replace(',', '').astype('Int64')

country_data

,Country,Land Area(Km2)
0,Afghanistan,652230
1,Albania,28748
2,Algeria,2381741
4,Angola,1246700
5,Antigua and Barbuda,443
...,...,...
190,Venezuela,912050
191,Vietnam,331210
192,Yemen,527968
193,Zambia,752618


In [162]:
cum_disasters = []
cum_deaths = []
for country in list(country_data['Country']):
    cum_disasters += [df['Country'].value_counts()[country]]
    cum_deaths += [sum(df.loc[df['Country']==country, 'Total Deaths'])]

country_data['Total number of disasters'] = cum_disasters
country_data['Total number of Deaths'] = cum_deaths
country_data['Num of disasters/area'] = country_data['Total number of disasters']/country_data['Land Area(Km2)']

country_data

,Country,Land Area(Km2),Total number of disasters,Total number of Deaths,Num of disasters/area
0,Afghanistan,652230,212,18232.0,0.000325
1,Albania,28748,30,513.0,0.001044
2,Algeria,2381741,101,5141.0,0.000042
4,Angola,1246700,100,5762.0,0.00008
5,Antigua and Barbuda,443,3,1.0,0.006772
...,...,...,...,...,...
190,Venezuela,912050,67,1253.0,0.000073
191,Vietnam,331210,227,6302.0,0.000685
192,Yemen,527968,91,3863.0,0.000172
193,Zambia,752618,69,1526.0,0.000092


In [121]:
df

,DisNo.,Disaster Group,Disaster Subgroup,Disaster Type,Country,Subregion,Region,Start Year,Total Deaths
0,1999-9388-DJI,Natural,Climatological,Drought,Djibouti,Sub-Saharan Africa,Africa,2001,0.0
1,1999-9388-SDN,Natural,Climatological,Drought,Sudan,Northern Africa,Africa,2000,0.0
2,1999-9388-SOM,Natural,Climatological,Drought,Somalia,Sub-Saharan Africa,Africa,2000,21.0
3,2000-0001-AGO,Technological,Transport,Road,Angola,Sub-Saharan Africa,Africa,2000,14.0
4,2000-0002-AGO,Natural,Hydrological,Flood,Angola,Sub-Saharan Africa,Africa,2000,31.0
...,...,...,...,...,...,...,...,...,...
15779,2024-9075-COL,Natural,Climatological,Drought,Colombia,Latin America and the Caribbean,Americas,2024,0.0
15780,2024-9131-ZMB,Natural,Climatological,Drought,Zambia,Sub-Saharan Africa,Africa,2024,0.0
15781,2024-9135-MHL,Natural,Climatological,Drought,Marshall Islands,Micronesia,Oceania,2024,0.0
15782,2024-9197-MWI,Natural,Climatological,Drought,Malawi,Sub-Saharan Africa,Africa,2024,0.0


In [163]:
# save the DataFrames to a new CSV files
df.to_csv('emdat_disasters_cleaned.csv', index=False, mode='w', encoding='utf-8')
country_data.to_csv('country_data_processed.csv', index=False, mode='w', encoding='utf-8')


# Todo

<input disabled="" type="checkbox"> Check outliers

<input disabled="" type="checkbox"> Check unknown chars in encoding

<input disabled="" type="checkbox"> Fill the columns 'Adjusted' when the value is not null

<input disabled="" type="checkbox"> Check for other columns to remove
